In [12]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State
import base64

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

import plotly.graph_objects as go
#import folium

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
exec(open('CRUDv2.py').read())





###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "admin"
password = "password"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.readAll('null','null'))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#

app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard Powers'))),
    html.Hr(),
    html.Div(
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
    ),
   
    html.Hr(),
    html.Center(html.Img(id='customer-image',src='data:image/png;base64,{}'.format(encoded_image.decode()),alt='customer image')),
    dcc.RadioItems(
                          id='filter-type',
                          # created the labels and keys based on the Grazioso requirements
                          options=[
                              {'label': 'Water Rescue', 'value': 'WR'},
                              {'label': 'Mountain/Wilderness Rescue', 'value': 'MWR'},
                              {'label': 'Disaster Rescue/Individual Tracking', 'value': 'DRIT'},
                              {'label': 'Reset - returns unfiltered state', 'value': 'RESET'}
                          ],
                          #defaults to reset option
                          value='RESET',
                          labelStyle={'display': 'inline-block'}
    ),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        data=df.to_dict('records'),
        editable=False,
        #filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable=False,
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current= 0,
        page_size= 10
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
        style={'display' : 'flex'},
            children=[
                #calls the graph
                html.Div(
                    dcc.Graph(id = 'graph-id'),
                    className='col s12 m6',
                ),
                html.Div(
                    #calls the map
                    dcc.Graph(id='map-id'),
                    className='col s12 m6',
                )
            ])
]) #end of app layout block

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(value):
### FIX ME Add code to filter interactive data table with MongoDB queries
    if value == 'RESET':
        df = pd.DataFrame(list(shelter.readAll('null','null')))
    elif value == 'WR':
        df = pd.DataFrame(list(shelter.readFilter('breed',{'$in':['Labrador Retriever Mix',
                                                                  'Chesapeake Bay Retriever',
                                                                  'Newfoundland']},
                                                  'sex_upon_outcome','Intact Female',
                                                  'age_upon_outcome_in_weeks',{'$gte':26},
                                                  'age_upon_outcome_in_weeks',{'$lte':156})))
    elif value == 'MWR':
        df = pd.DataFrame(list(shelter.readFilter('breed',{'$in':['German Shepherd',
                                                                  'Alaskan Malamute',
                                                                  'Old English Sheepdog',
                                                                  'Siberian Husky',
                                                                  'Rottweiler']},
                                                 'sex_upon_outcome','Intact Male',
                                                 'age_upon_outcome_in_weeks',{'$gte':26},
                                                 'age_upon_outcome_in_weeks',{'$lte':156})))
    elif value == 'DRIT':
        df = pd.DataFrame(list(shelter.readFilter('breed',{'$in':['Doberman Pinscher',
                                                                  'German Shepherd',
                                                                  'Golden Retriever',
                                                                  'Bloodhound',
                                                                  'Rottweiler']},
                                                 'sex_upon_outcome','Intact Male',
                                                 'age_upon_outcome_in_weeks',{'$gte':20},
                                                 'age_upon_outcome_in_weeks',{'$lte':300})))
        
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data=df.to_dict('records')
        
        
    return data,columns




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

#graph definition

@app.callback(
    Output('graph-id', "figure"),
    [Input('datatable-id', "data")])
def update_graphs(data):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) 
    dff = pd.DataFrame.from_dict(data)

    fig = px.pie(
        dff,
        names='breed'
    )

    return fig

@app.callback(
    Output('map-id', "figure"),
    [Input('datatable-id', "data")])
#FIXME: Add in the code for your geolocation chart
#If you completed the Module Six Assignment, you can copy in the code you created here.

def update_map(data):
#FIXME Add in the code for your geolocation chart
    #pull the information from the table through the viewdata variable
    dff = pd.DataFrame.from_dict(data)
    
    #set a selected animal
    #elected_animal = None
    #f not bool(derived_virtual_selected_rows):
    #   selected_animal = dff.loc[0]
    #lse:
    #   selected_animal = dff.loc[derived_virtual_selected_rows]
    
    #set the variables of the selected animal
    #latitude = dff.loc[0,'location_lat']
    #longitude = dff.loc[0,'location_long']
    #breed = dff.loc[0,'breed']
    #name = dff.loc[0,'name']
    
    #set size of map
    #create map using folium library
    #dogMap = folium.Map(location=[30.75, -97.48], zoom_start=12)
    #toolTip = breed
    #marker = folium.Marker(
    #    location = [latitude,longitude],
    #    popup = '<strong>Dog</strong>',
    #    tooltip = toolTip
    #)
    #marker.add_to(dogMap)
    
    dogMap = px.scatter_mapbox(dff,
                               lat = 'location_lat',
                               lon = 'location_long',
                               hover_name = 'name',
                               hover_data=['breed', 'outcome_type'],
                               zoom = 8)
    dogMap.update_layout(mapbox_style="open-street-map")
    dogMap.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    
    #dogMap = dl.Map(style = {'width':'1000px','height':'500px'},
    #                #centered on Austin, TX
    #                center = [30.75,-97.48],
    #                dl.TileLayer(id = 'base-layer-id'),
    #                dl.Marker(position = [latitude,longitude], children = [
    #                    dl.Tooltip(breed),
    #                    dl.Popup([
    #                        html.H1('Animal Name'),
    #                        html.P(name)
    #                    ])
    #                ])
    #               )
    
    return dogMap
        

app